In [4]:
!apt-get install libxslt1-dev libxml2 python-dev
!sudo pip3 install bs4
!sudo pip3 install lxml

# 注意：cgi 版程式只能在 apache2 伺服器或其他完成 cgi-bin 設置的伺服器上執行。參考網址如下：
http://apps.nknush.kh.edu.tw:8000/cgi-bin/findBook.py?keyword=三國

In [ ]:
#!/usr/bin/python3

# 作為一個 python3 的 cgi ，必須改寫 sys.stdout。

import requests
import re
import json
from bs4 import BeautifulSoup
import sys
import io
import cgi, cgitb 

sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')

# 存取網址 http://apps.nknush.kh.edu.tw:8000/cgi-bin/findBook.py?keyword=三國
form = cgi.FieldStorage() 
keyword = form.getvalue('keyword')
formdata={
    'ToolkitScriptManager1_HiddenField':'',
    '__EVENTTARGET':'ctl00$ContentPlaceHolder1$UcRankLoan$lbtn_Rank_title',
    '__EVENTARGUMENT':'',
    '__VIEWSTATE':'...',
    '__VIEWSTATEGENERATOR':'4DE51EF8',
    '__VIEWSTATEENCRYPTED':'',
    '__EVENTVALIDATION':'aeb/7Iuita0LXioaoZNQhUhHttSj/rXCRlLPZd1YKxTWlOMhF2/Qa0p1jrfN+1kfeY3lIkf01JGQI2s7GTR4UcjSbiOtgop5zdAGq9++ufpXCC70biLJ2pDWiWGnVLrYADE1ASlILP+L893t+QQH4Avj1zWx2O0M/y3+HxE54HKvuOEPmbO06WWrcxNkDjU9nPrlwcnyLLKEx/Us3VY86pcJbvm4D0XtXD1tEK3hXEW8B3p1GJjnBmqbu4lQeSf76BvRVfBpNT4/465suG+SleMySr0DjlxNlQ3fL0VpZPVPPWBMWtJz8MkHtTK7gXFz4bdN6N9ssgleBl4Mkp8C8ajR4GMiCF0UukO1tas+J31IkO37Vhkyl2kAJ8vveqplsnYg8jJPr21Mnul+MHzc6jLa53glHGvkKweb7/pv3vspFtyU7J/tt1ejBot6jxt4OdKnWu/VbQQd9W4CXgnHMy7aL0TKy+NuOuJ5ycXquaWFcaIHwg3H0koaKAGzTIW0/6h85D38Kw27wv300zCJ+V7F22nBfQT+kbbghq4ttbJh4qrgdeUMhfmTPeSs+8LgpuXVTpagYzxUZ2H+x3E22gN68M6gi4L6Drr11LXs1hAARUI6T/yTx2VB5W9lCP1h68Dud11M5qvWDegBgTIr62ZcLI+XwO7UGN9RYqxc/qI/bJVz96XuN9H8mgIOhGL22zKvaedv04/6a0pOLiRB1cZKV0A3ZsvFetLyZdLjxOUhXm2YBIXVfVAA7cyr4gKn+xGIVh/ufHsb/HjIC3FPaRudvlAwD0ekSUqom/c09AaWZ9XeC/w2mXwc8ooOghK+HWGWHLQeIq8Oal4PjdMiZEnz0iH/zggXYV53RGCc1M8W2RnQ60FOpcYTfp2irJ41LJbU3W4L9d2AH1nr4WcA00Om6K5aEfjiQKC5fQk/Vsi6sDMdH+RSmALqQ4hDvTxNBpnHskNGDPChGIO/Clx4dgt1HllLfye+w2z6nFuEv4Xaf6GH3PXKqs1Df9KoQ8zSjiySHLoO8+cqvJDFIvi4UKBC3aMhLzvvbkm0/YQyUck98XlwD05q2Nrm9bRdR+SJX2SMkjW2k5trYsnhWQlv0Kr7OEDm9E13rc3bmcd8ffvL2jKXXZrdKpWWUA9mRsCv3P2NVGoS6BxbqlUoxmRxJw4TjmQLye342GiiyYPIYwthEY6qgZZibQeIAWIIahOYhSss67C4P6Cki62/ErpfPp2HO1C6vi5BkCFQoSaHOvPhMpNw0WmB+vFdf8NhOr8bOfEY0+tve8Lr9kB+stoy1RWQeYBKg0q3JQRxhcvPIrauODS+moc79xZV9H9vObEJ66IRPbK5/t94gBrbhzF7qA==',
    'ctl00$ContentPlaceHolder1$hid_Right':'1',
    'ctl00$ContentPlaceHolder1$UcQuickSearch$hid_SearchTab_Index':'0',
    'ctl00$ContentPlaceHolder1$UcQuickSearch$txt_SearchTab':keyword,
    'ctl00$ContentPlaceHolder1$UcQuickSearch$btn_SearchTab':'搜尋'
}
res = requests.post("http://163.32.92.129/OPAC850/", data=formdata)
soup = BeautifulSoup(res.text, 'lxml')
list = []
for item in soup.find_all("tr", id=re.compile("(.*)ProductsRow$")):
    #print("item=",item) # 取消註解觀察抓到的資料。    
    trtrtdi = 0
    dic = {}
    for trtr in item.find_all("tr"):
        for trtrtd in trtr.find_all("td"):
            #trtrtd = ' '.join(trtrtd.text.split()).split()
            trtrtd = trtrtd.text.strip()
            trtrtd = ' '.join(trtrtd.split()) # 由於資料當中有許多不必要的空白，因此切斷後補空白再接起來。
            #print("trtrtd",trtrtdi,"=", trtrtd)
            if trtrtdi == 1:
                dic['bookname'] = trtrtd
            if trtrtdi == 3:
                dic['author'] = trtrtd
            if trtrtdi == 4:
                dic['publisher'] = trtrtd
            if trtrtdi == 5:
                dic['year'] = trtrtd
            trtrtdi = trtrtdi + 1
    list.append(dic)

print ("Content-type:text/html")
print ()
print(json.dumps(list,ensure_ascii=False))
